pip install -q -U google-generativeai <br>
pip install python-dotenv

In [72]:
# Import the Python SDK
import google.generativeai as genai  # type: ignore
from dotenv import load_dotenv
import os
import json
import random

# Load environment variables from the .env file
load_dotenv()

# Access the API key
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

genai.configure(api_key=GEMINI_API_KEY)

## Initialize the Generative Model
model = genai.GenerativeModel('gemini-pro')

In [73]:
# Load employees from a JSON file
employees_file = "assets/EmployeeInfo.json"
if os.path.exists(employees_file):
    with open(employees_file, "r") as file:
        employees = json.load(file)
else:
    raise FileNotFoundError(f"Employee file '{employees_file}' not found.")


In [54]:
# Example phishing parameters
phishing_examples = [
    {"Reason": "Account Suspicious Activity", "Fake Link": "https://example.com/secure-login", "Created By": "Sam Sussy"},
    {"Reason": "Password Expiry Notification", "Fake Link": "https://example.com/reset-password", "Created By": "Sally Sneaky"},
    {"Reason": "Exclusive Training Webinar", "Fake Link": "https://example.com/join-webinar", "Created By": "Richard Rascal"},
    {"Reason": "Email Storage Full", "Fake Link": "https://example.com/manage-storage", "Created By": "Bernard Bandit"}
]

In [74]:
# Predefined lists of components for phishing examples
reasons = [
    "Account Suspicious Activity",
    "Password Expiry Notification",
    "Exclusive Training Webinar",
    "Email Storage Full",
    "Important Security Update",
    "New Policy Acknowledgment Required",
    "Urgent Billing Issue",
    "System Maintenance Notification"
]

fake_domains = [
    "secure-login", "reset-password", "join-webinar", "manage-storage",
    "update-profile", "confirm-identity", "billing-alert", "policy-update"
]

creators = ["Sam Sussy", "Sally Sneaky", "Richard Rascal", "Bernard Bandit", 
            "Tim Tricky", "Nancy Notorious", "Fred Fraudster", "Holly Hoaxer"]

# Base domain for fake links
base_domain = "https://example.com/"

# Function to generate random phishing examples
def generate_phishing_examples(num_examples=5):
    phishing_examples = []
    for _ in range(num_examples):
        reason = random.choice(reasons)
        fake_link = base_domain + random.choice(fake_domains)
        created_by = random.choice(creators)
        phishing_examples.append({
            "Reason": reason,
            "Fake Link": fake_link,
            "Created By": created_by
        })
    return phishing_examples

# Generate a dynamic list of phishing examples
phishing_examples = generate_phishing_examples(10)

# Print the generated examples
for example in phishing_examples:
    print(example)


{'Reason': 'Important Security Update', 'Fake Link': 'https://example.com/secure-login', 'Created By': 'Sally Sneaky'}
{'Reason': 'System Maintenance Notification', 'Fake Link': 'https://example.com/policy-update', 'Created By': 'Bernard Bandit'}
{'Reason': 'Account Suspicious Activity', 'Fake Link': 'https://example.com/confirm-identity', 'Created By': 'Tim Tricky'}
{'Reason': 'Exclusive Training Webinar', 'Fake Link': 'https://example.com/reset-password', 'Created By': 'Bernard Bandit'}
{'Reason': 'Important Security Update', 'Fake Link': 'https://example.com/manage-storage', 'Created By': 'Sally Sneaky'}
{'Reason': 'Email Storage Full', 'Fake Link': 'https://example.com/join-webinar', 'Created By': 'Sally Sneaky'}
{'Reason': 'Important Security Update', 'Fake Link': 'https://example.com/policy-update', 'Created By': 'Bernard Bandit'}
{'Reason': 'Exclusive Training Webinar', 'Fake Link': 'https://example.com/join-webinar', 'Created By': 'Sally Sneaky'}
{'Reason': 'Email Storage Full'

In [75]:
# Function to generate a modular prompt
def generate_prompt(employee, random_pick):
    return f"""
    Write a concise and professional email from {random_pick['Created By']} to {employee['FirstName']} {employee['LastName']} from the {employee['Role']} team.
    Focus on the theme of their role as a {employee['Role']}.
    Include this link: {random_pick['Fake Link']} for a specific call to action related to their role.
    The email should be brief, similar in length to the following example:
    "Hello [Name],
    We have rolled out new features on the analytics dashboard to enhance data analysis efficiency. Please review these updates and confirm functionality by accessing the following link:
    [Review Dashboard Features Here]
    Thank you,
    Data Analytics Team"
    Maintain a professional tone, avoid urgency, and ensure the link is prominent but natural.
    Only provide the body of the email, no subject or signature.
    """

# Generate a phishing email for a randomly selected employee
random_employee = random.choice(employees)
random_pick = random.choice(phishing_examples)
prompt = generate_prompt(random_employee, random_pick)

# API call to generate the email body
body_response = model.generate_content(prompt)
email_body = body_response.text

# API call to generate the email subject line
subject_prompt = f"Write a compelling, realistic email subject line for the following email:\n{email_body}"
subject_response = model.generate_content(subject_prompt)
email_subject = subject_response.text

# Output the generated email
print("Subject:", email_subject)
print("\n"  + email_body)
   

Subject: Important AI Policy Updates for Proximus Ada

Hi BenoÃ®t,

As the Head of AI at Proximus Ada, you play a vital role in shaping the future of AI within the organization. To support you in this, we have released some updates to our AI policy that you may find valuable: https://example.com/policy-update

Please take some time to review these updates and ensure that your team is compliant. This will help align Proximus Ada's AI practices with industry best practices and ethical guidelines.

Thank you for your attention to this matter.


In [82]:
# Dynamic explanation generator
def generate_explanation(employee, random_pick):
    explanation = (
        f"* **Generic Greeting:** The email uses 'Hi {employee['FirstName']}' instead of a more formal address with a last name, "
        "which is common in phishing emails.\n"
        "* **Unsolicited Contact:** Claims to address the recipient about a topic ({random_pick['Reason']}) without prior interaction.\n"
        f"* **Unverified Company Name:** The sender, {random_pick['Created By']}, is not recognizable, raising suspicions.\n"
        f"* **Suspicious Link:** The link ({random_pick['Fake Link']}) does not correspond to a trusted company domain and could lead to a phishing site.\n"
        "* **Pressure to Act:** Encourages the recipient to review or act quickly, creating subtle urgency.\n"
        "* **Vague Context:** Does not provide specific, verifiable details about the supposed issue or request."
    )
    return explanation

# Function to generate phishing email JSON data for employees
def generate_phishing_emails(employees, model, num_examples=5):
 
    phishing_emails = []
    for employee in employees:
        for _ in range(num_examples):
           
            random_pick = random.choice(phishing_examples)
            prompt = generate_prompt(employee, random_pick)

            # Generate email body using the model
            body_response = model.generate_content(prompt)
            email_body = body_response.text

            # Generate email subject using the model
            subject_prompt = f"Write a compelling, realistic email subject line for the following email:\n{email_body}"
            subject_response = model.generate_content(subject_prompt)
            email_subject = subject_response.text

            explanation = generate_explanation(employee, random_pick)

            phishing_email = {
                "FirstName": employee["FirstName"],
                "LastName": employee["LastName"],
                "Role": employee["Role"],
                "Email": employee["Email"],
                "body": email_body,
                "subject": email_subject,
                "explanation": explanation
            }

            phishing_emails.append(phishing_email)

    return phishing_emails

In [81]:

phishing_emails = generate_phishing_emails(employees, model, num_examples=1)

# Save the generated emails to a JSON file
output_file = "phishing_emails.json"
with open(output_file, "w") as file:
    json.dump(phishing_emails, file, indent=4)

print(f"Phishing emails saved to {output_file}")

Phishing emails saved to phishing_emails.json
